In [1]:
%load_ext autoreload
%autoreload 2

## Pytorch EO Tasks

Pytorch EO core design revolves around `Tasks`. Examples of tasks are: image classification, object detection, image segmentation, etc.

A task is defined by:
1. `model`: a neural network that will be trained to perform the task
2. `hparams`: hyperparameters used to train the model (optimizer, scheduler and additional information)
3. `loss function`: criterion used during the optimization process
4. `metrics`: evaluation of the model in the task
5. `inputs` and `outputs`: lists with the names of the inputs-outputs
6. `other parameters`: additional parameters that are task specific

You don't have to provide all these paremeters, we offer good defaults for most of them. However, you can customize them as you wish.

## Models

Models are neural networks that take inputs and produce outputs. You can build your own or use third party models. By default we use `torchvision`, so you can use any model from there

In [2]:
import torch 
from pytorch_eo.tasks.BaseTask import BaseTask

task = BaseTask('resnet18')

output = task(torch.randn(32,3,224,224))
output.shape

torch.Size([32, 1000])

In case you use models from torchvision, you can pass any parameter in the `hparams` object as follows.

In [3]:
hparams = {
    'model': {
        'weights': 'ResNet18_Weights.IMAGENET1K_V1'
    }
}

task = BaseTask('resnet18', hparams)

output = task(torch.randn(32,3,224,224))
output.shape

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/juan/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 63.9MB/s]


torch.Size([32, 1000])

You can always make your own models, for example modifying an existing model from torchvision.

In [4]:
import torch
import torchvision 

model = torchvision.models.resnet18()
model.fc = torch.nn.Linear(512, 10)

task = BaseTask(model)

output = task(torch.randn(32,3,224,224))
output.shape

torch.Size([32, 10])

Or building your own model from scratch.

In [5]:
class MyModel(torch.nn.Module):
    def __init__(self, pretrained, num_classes):
        super().__init__()
        resnet = torchvision.models.resnet18(pretrained=pretrained)
        self.backbone = torch.nn.Sequential(*list(resnet.children())[:-2])
        self.fc = torch.nn.Sequential(
            torch.nn.AdaptiveAvgPool2d((1, 1)),
            torch.nn.Flatten(),
            torch.nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        f = self.backbone(x)
        return self.fc(f)

In [6]:
model = MyModel(pretrained=True, num_classes=10)

task = BaseTask(model)

output = task(torch.randn(32, 3, 224,224))
output.shape

/home/juan/Desktop/pytorch_eo/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/juan/Desktop/pytorch_eo/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([32, 10])

But you would probably want to use third party implementations with extra functionality.

In [7]:
import timm 

model = timm.create_model(
    'resnet18',
    pretrained='imagenet',
    in_chans=3,
    num_classes=10,
    features_only=True
)

task = BaseTask(model)

output = task(torch.randn(32,3,224,224))
for o in output:
    print(o.shape)

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

torch.Size([32, 64, 112, 112])
torch.Size([32, 64, 56, 56])
torch.Size([32, 128, 28, 28])
torch.Size([32, 256, 14, 14])
torch.Size([32, 512, 7, 7])


In [8]:
import segmentation_models_pytorch as smp

model = smp.Unet(
    encoder_name='resnet18',
    encoder_weights='imagenet',
    in_channels=3,
    classes=2,
)

task = BaseTask(model)

output = task(torch.randn(32,3,224,224))
output.shape

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/juan/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 53.5MB/s]


torch.Size([32, 2, 224, 224])

## hparams

This is a `dict` holding the hyperparameters used for training. Pytorch lightning will save the object in the model's checkpoint (that can be used to resume training, for example) so make sure to add any additional information that you want to save. In some cases, default parameters will be used if hparams are not provided.

In [9]:
hparams = {
    # hparams used for training
    'model': { # only if you use default torchvision models
        'weights': 'ResNet18_Weights.IMAGENET1K_V1'
    },
    'loss': 'CrossEntropyLoss', # choose one from pytorch docs
    'loss_params': {}, 
    'optimizer': 'Adam', # choose one from pytorch docs
    'optim_params': {
        'lr': 1e-4
    },
    'scheduler': 'CosineAnnealingLR', # choose one from pytorch docs
    'scheduler_params': {
        'T_max': 10,
        'verbose': True
    }
    # extra
    # epochs, batch size, model, transforms, ...
}

## Loss Function

You can specify the loss function to use during training as follows

In [10]:
import lightning as L
from pytorch_eo.datasets import EuroSATRGB
from pytorch_eo.tasks import ImageClassification
import albumentations as A
from albumentations.pytorch import ToTensorV2

trans = A.Compose([
    A.Normalize(0, 1),
    ToTensorV2(),
])

ds = EuroSATRGB(batch_size=32, train_trans=trans, val_trans=trans)

model = torchvision.models.resnet18()
model.fc = torch.nn.Linear(512, 10)

task = ImageClassification(model, loss_fn=torch.nn.CrossEntropyLoss(), num_classes=ds.num_classes)

trainer = L.Trainer(
    accelerator='cuda',
    devices=1,
    precision=16,
    max_epochs=3,
    limit_train_batches=10
)

trainer.fit(task, ds)

/home/juan/Desktop/pytorch_eo/.venv/lib/python3.12/site-packages/lightning/fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/juan/Desktop/pytorch_eo/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install ligh

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/juan/Desktop/pytorch_eo/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/juan/Desktop/pytorch_eo/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/juan/Desktop/pytorch_eo/.venv/lib/python3.12/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


## Metrics

The last piece to train a model is the metrics. You can train without metrics, use third party libraries like `torchmetrics` or use your own implementations. In most tasks we will use at least one metric even if they are not provided (they are that important !)

In [11]:
import pytorch_lightning as pl

from pytorch_eo.datasets import EuroSATRGB
from pytorch_eo.tasks import ImageClassification
import torchmetrics

ds = EuroSATRGB(batch_size=32, train_trans=trans, val_trans=trans)

model = torchvision.models.resnet18()
model.fc = torch.nn.Linear(512, ds.num_classes)

metrics = {'acc': torchmetrics.Accuracy(task="multiclass", num_classes=ds.num_classes)}

task = ImageClassification(model, metrics=metrics)

In [12]:
trainer = L.Trainer(
    accelerator='cuda',
    devices=1,
    precision=16,
    max_epochs=3,
    limit_train_batches=10
)

trainer.fit(task, ds)

/home/juan/Desktop/pytorch_eo/.venv/lib/python3.12/site-packages/lightning/fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type               | Params | Mode 
-------------------------------------------------------
0 | model   | ResNet             | 11.2 M | train
1 | loss_fn | CrossEntropyLoss   | 0      | train
2 | acc     | MulticlassAccuracy | 0      | train
-------------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)
70        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/juan/Desktop/pytorch_eo/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/juan/Desktop/pytorch_eo/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/juan/Desktop/pytorch_eo/.venv/lib/python3.12/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


You can use as many metrics as you want (all will be logged)

In [13]:
def my_accuracy(y_hat, y):
    return (torch.argmax(y_hat, axis=1) == y).sum() / y.shape[0]

metrics = {
    'acc': torchmetrics.Accuracy(task="multiclass", num_classes=ds.num_classes), 
    'my_acc': my_accuracy
}

task = ImageClassification(model, hparams, metrics=metrics)

In [14]:
trainer = L.Trainer(
    accelerator='cuda',
    devices=1,
    precision=16,
    max_epochs=3,
    limit_train_batches=10
)

trainer.fit(task, ds)

/home/juan/Desktop/pytorch_eo/.venv/lib/python3.12/site-packages/lightning/fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/juan/Desktop/pytorch_eo/.venv/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(

  | Name    | Type               | Params | Mode 
-------------------------------------------------------
0 | model   | ResNet             | 11.2 M | train
1 | loss_fn | CrossEntropyLoss   | 0      | train
2 | acc     | MulticlassAccuracy | 0      | train
-------------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)
70        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/juan/Desktop/pytorch_eo/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/juan/Desktop/pytorch_eo/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/juan/Desktop/pytorch_eo/.venv/lib/python3.12/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


## Inputs and Outputs

PytorchEO datasets are designed to be flexible in the number and modality of inputs and outputs in order to allow for advanced applications such as data fusion, unsupervised learning or multi-task learning. For this reason, you must tell the task which fields in your datasets have to be used for inputs and outputs, respectively.

In [15]:
import os
from pytorch_eo.utils import download_url, unzip_file
from pathlib import Path
from pytorch_eo.datasets import RGBImageDataset
from pytorch_eo.datasets import ConcatDataset
from torch.utils.data import DataLoader
import lightning as L
from torch.utils.data import DataLoader

def prepare_data(path):
	os.makedirs(path, exist_ok=True)
	url = "http://madm.dfki.de/files/sentinel/EuroSAT.zip"
	compressed_data_filename = 'EuroSAT.zip'
	data_folder = '2750'
	compressed_data_path = path / compressed_data_filename
	download_url(url, compressed_data_path)
	unzip_file(compressed_data_path, path, msg="extracting data ...")
	uncompressed_data_path = path / data_folder
	classes = sorted(os.listdir(uncompressed_data_path))
	images, labels = [], []
	for ix, label in enumerate(classes):
		_images = os.listdir(uncompressed_data_path / label)
		images += [str(uncompressed_data_path /
					label / img) for img in _images]
		labels += [ix]*len(_images)
	return images, labels

class EuroSATDataset(L.LightningDataModule):
	def __init__(self, path='data', trans=None):
		super().__init__()
		self.path = Path(path)
		self.trans = trans

	def setup(self, stage=None):
		images, labels = prepare_data(self.path)
		images_ds = RGBImageDataset(images) 
		# Custom keys in the dataset !
		self.ds = ConcatDataset({'abc': images_ds, 'def': labels}, trans=self.trans, image_key='abc')

	def train_dataloader(self):
		return DataLoader(self.ds)

In [16]:
trans = A.Compose([
    A.Normalize(0, 1),
    ToTensorV2(),
], additional_targets={'abc': 'image'})
ds = EuroSATDataset(trans=trans)
model = torchvision.models.resnet18()
model.fc = torch.nn.Linear(512, 10)

task = ImageClassification(model, inputs=['abc'], outputs=['def'], num_classes=10)

trainer = L.Trainer(
    accelerator='cuda',
    devices=1,
    precision=16,
    max_epochs=3,
    limit_train_batches=10
)

trainer.fit(task, ds)

/home/juan/Desktop/pytorch_eo/.venv/lib/python3.12/site-packages/lightning/fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/juan/Desktop/pytorch_eo/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
extracting data ...: 100%|██████████| 27011/27011 [00:03<00:00, 8470.04it/s]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type               | Params | Mode 
-------------------------------------------------------
0 | model   | ResNet             | 11.2 M | trai

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


## Tasks are `LightningModule`s

Finally, note that `Tasks` are built on top of `Pytorch Lightning`'s `LightningModule`, so you can build your own tasks easily and use any of our `Datasets` (which are in turn `LightningDataModule`s).